## Creacion de formatos de tablas

El objetivo aqui es crear un conjunto de query para la creacion, insercion y agregacion de foreign key de tabla referidas a columnas que tienen mas de 1 dimension categorica. 

In [1]:
import pandas as pd 
pd.set_option('display.max_rows', None)

Exploracion de las hojas del xlsx, el cual en su estructura contiene en nombre de las columnas de nuestra db, los datos categoricos de esas columnas, una descripción de estas y ademas se agrego el nombre de la tabla a la que se puede hacer referencia (creada), dejando sin dato las columnas que no tendran este tipo de tabla

In [44]:

ruta_xlsx = "./in/04_Diccionario_Variables_ENL2022_2.xlsx"
ruta_csv = './in/V_ENL2022_300_400.csv'

# Cargar todas las hojas del archivo
xls = pd.ExcelFile(ruta_xlsx)  

# Crear un diccionario para almacenar la primera columna de cada hoja
df_hojas = []

# Iterar sobre cada hoja
for hoja in xls.sheet_names:
    # Leer la hoja actual
    df = pd.read_excel(xls, sheet_name=hoja)
    df = df.fillna(0)
    df_hojas.append(df)



In [87]:
# Funcion para agregar los datos categoricos en tupla (formato que facilita el INSERT)
def split_data(values_table: list):
    split_values = values_table.split('\n')
    new_values = []
    for val in split_values:
        new_val = val[3:]
        # print(new_val)
        new_values.append(new_val)

    return  new_values
    

In [127]:
# Funcion que estructura la query de creacion, insercion y fk
def create_table(name_table: str, values_table: list, name_column: str):
    # Valores para insercion ------------------------------------
    values_tuble = [(0,'Valores Nulos')]

    for i in range(0,len(values_table)):
        tup = (i+1, values_table[i])
        values_tuble.append(tup)

    insert_values = str((values_tuble))[1:-1] # Quitamos los corchetes
    # -----------------------------------------------------------

    query = f""" ----------------------------------------------------
-- Nombre columna: {name_column}
-- Tabla {name_table}
CREATE TABLE public.{name_table} (
    id_{name_column} integer PRIMARY KEY,
    {name_column} VARCHAR(100)
);

-- Valores para insercion
INSERT INTO public.{name_table} VALUES 
{insert_values};

-- creamos la fk
ALTER TABLE public.data_total
ADD CONSTRAINT fk_{name_table}
FOREIGN KEY ({name_column})
REFERENCES public.{name_table} (id_{name_column});
"""

    return query

Codigo  que genera las querys para todas las tablas que fueron referidas 

In [128]:
import numpy as np 

names_sheets = xls.sheet_names
posicion_sheet = 0

for df_pr in df_hojas:
    rows,cols = df_pr.shape
    print(f'-- {names_sheets[posicion_sheet]}') # Nombre de Hoja
    posicion_sheet += 1

    for row in range(0, rows):
        name_table = df_pr.iloc[row,cols-2]
        values_table = df_pr.iloc[row,cols-1]
        name_col = df_pr.iloc[row,0]

        if name_table != 0:
            #print(name_table)
            #print(name_col.lower())
            #print(split_data(values_table))
            #print()
            query = create_table(name_table, split_data(values_table), name_col.lower())
            print(query)


            print(' ')

-- GENERALES
 ----------------------------------------------------
-- Nombre columna: dominio
-- Tabla dominio_geografico
CREATE TABLE public.dominio_geografico (
    id_dominio integer PRIMARY KEY,
    dominio VARCHAR(100)
);

-- Valores para insercion
INSERT INTO public.dominio_geografico VALUES 
(0, 'Valores Nulos'), (1, 'Costa Norte'), (2, 'Costa Centro'), (3, 'Costa Sur'), (4, 'Sierra Norte'), (5, 'Sierra Centro'), (6, 'Sierra Sur'), (7, 'Selva'), (8, 'Lima Metropolitana');

-- creamos la fk
ALTER TABLE public.data_total
ADD CONSTRAINT fk_dominio_geografico
FOREIGN KEY (dominio)
REFERENCES public.dominio_geografico (id_dominio);

 
 ----------------------------------------------------
-- Nombre columna: estrato
-- Tabla estrato_geografico
CREATE TABLE public.estrato_geografico (
    id_estrato integer PRIMARY KEY,
    estrato VARCHAR(100)
);

-- Valores para insercion
INSERT INTO public.estrato_geografico VALUES 
(0, 'Valores Nulos'), (1, 'De 500 000 a más habitantes.'), (2, 'De 1

## NOTA

Si bien se debieron realizar algunas modificaciones de ediccion como modificacion en los datos de insercion o problemas de tipado por la lectura del xlsx, el resto de codigo funciono perfectamente por lo cual se puede tomar el procedimiento como exitoso a un 99%